In [2]:
pip install python-docx

In [6]:
import pandas as pd
from docx import Document


def read_docx(file_path):
    doc = Document(file_path)
    data = []

    for table in doc.tables:
        # sets the first row as headers
        headers = [cell.text for cell in table.rows[0].cells]
        for row in table.rows[1:]:
            data.append([cell.text for cell in row.cells])

    return pd.DataFrame(data, columns=headers)

# Replace 'path_to_docx_file' with your actual file path
file_path = 'employee.docx'
df = read_docx(file_path)
df

,Employee ID,First Name,Last Name,Contact Number,Gender
0,e00001,Christopher,Thomas,7123456790,Male
1,e00002,Isabella,Collins,9123456788,Female
2,e00003,James,Allen,3123456788,Male
3,e00004,Elizabeth,Morris,4123456790,Female
4,e00005,David,Clark,6123456790,Male
5,e00006,Sophia,Young,8123456788,Female
6,e00007,Matthew,Adams,2123456790,Male
7,e00008,Olivia,Hall,5123456790,Female
8,e00009,Ethan,Russell,7123456785,Male
9,e00010,Emma,Watson,9123456786,Female


Employee ID       object
First Name        object
Last Name         object
Contact Number    object
Gender            object
dtype: object

In [4]:
## Customer.csv
import pandas as pd
customer = pd.read_csv('customer.csv')

In [2]:
# checking for missing data 
missing = customer.isna().sum()
print(missing)

Customer ID       0
First Name        0
Last Name         0
Company Name      0
Contact Number    0
dtype: int64


In [7]:
# checking for duplicates 
duplicates = customer.duplicated().sum()
print(duplicates)

0
